In [ ]:
## hhayn router1
from typing_extensions import Literal
from langgraph.graph import MessagesState

# =========================
# 1. State 정의
# =========================
class Router1State(MessagesState):
    # 사용자 입력
    question: str

    # HR 카테고리 결과
    # none: HR 무관
    # work: 근무 / leave: 휴가 / equipment: 장비보안 / welfare: 복지 / etc: 급여·비용·출장
    is_hr_question: Literal["none", "work", "leave", "equipment", "welfare", "etc"]

    # 라우팅 방향
    # HR 관련이면 rag, 무관하면 reject
    next_step: Literal["rag", "reject"]

# =========================
# 2. Router1 Node
# =========================
def router1_node(state: Router1State) -> Router1State:
    question = state["question"].lower()

    hr_keywords = {
        "work": ["근무", "출근", "퇴근", "재택", "야근", "코어타임"],
        "leave": ["연차", "휴가", "반차", "병가", "경조사"],
        "equipment": ["노트북", "장비", "보안", "카드키", "인증"],
        "welfare": ["복지", "포인트", "식대", "교육비", "상담"],
        "etc": ["급여", "월급", "비용", "출장", "정산", "식비"],
    }

    is_hr = "none"
    for category, keywords in hr_keywords.items():
        if any(kw in question for kw in keywords):
            is_hr = category
            break

    next_step = "rag" if is_hr != "none" else "reject"

    return {
        **state,
        "is_hr_question": is_hr,
        "next_step": next_step,
    }

# =========================
# 3. Router1 Router
# =========================
def route_after_router1(state: Router1State) -> str:
    if state["next_step"] == "rag":
        return "router2"   # HR 관련 질문이면 Router2로 이동
    else:
        return "reject"    # HR 무관 → Reject 노드로 이동

# =========================
# 4. Reject Node
# =========================
def reject_node(state: Router1State) -> dict:
    """
    HR 무관 질문에 대해 거절 안내 메시지를 생성하는 노드
    """
    return {
        **state,
        "answer": "❌ 지원하지 않는 질문입니다. HR 정책 관련 문의만 가능합니다."
    }